# RetinaNet
Based on: https://debuggercafe.com/object-detection-using-retinanet-with-pytorch-and-deep-learning/ <br>
**Is recommendend to use a virtual environment because packages instalation will be neccesary**

This notebook assumes:
* That this notebook is in a folder were you want to download the repositories (otherwise please indicate the path to it when is necessary)
* That this notebook is in the same folder as metadata.json and the folder with the images extracted

In [1]:
%pip install torch torchvision pillow tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\IRProject\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
# Libraries
from pathlib import Path
from PIL import Image
import cv2
import os
import random
import numpy as np
import pandas as pd
import keras_resnet

In [2]:
# Basics paths
base_path = Path(os.path.dirname(os.path.realpath("__file__")))  # Where is located this jupyter file
images_dir_path = base_path / "images"                           # Change this if the images folder is in another place 
metadata_path = base_path / "metadata.json"
print(base_path)

D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project


In [3]:
# Create RetinaNet directory
Path(base_path / "RetNet").resolve().mkdir(parents=True, exist_ok=True)
retnet_path = base_path / "RetNet"

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Cells to run once!!!

In [4]:
# Install the clone new folder (run this once)
%cd RetNet
if 'keras-retinanet' in os.listdir('.'):
    pass
else:
    !git clone https://github.com/fizyr/keras-retinanet
%cd ..

D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project\RetNet
D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project


In [5]:
# Install the clone new folder (run this once)
%cd RetNet
if 'Retinanet-Tutorial' in os.listdir('.'):
    pass
else:
    !git clone https://github.com/jaspereb/Retinanet-Tutorial
%cd ..

D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project\RetNet
D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project


In [14]:
# Compile the repository (run this once)
%cd RetNet/keras-retinanet
!pip install . --user yes
%cd ../..

D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project\RetNet\keras-retinanet
^C
D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing d:\avelyn\avytareas\dsleuven\advanced_analytics_in_business\assignment_2\project\retnet\keras-retinanet
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp39-cp39-win_amd64.whl size=128833 sha256=e5f928e225125ffc690a97fea9121926c7c6c407e62f0c14cde1848c84b1680d
  Stored in directory: C:\Users\Avelyn\AppData\Local\Temp\pip-ephem-wheel-cache-uimi2kp9\wheels\20\70\07\653e353016d8feba39901927f3bb8a3308d594768475b93921
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 1.0.0
    Uninstalling keras-retinanet-1.0.0:
      Successfully u

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


-----------------------------------------------------------------------------------------------------------------------------------

In [21]:
# Functions for later

# Function necessary to create the labels txt
def get_squares(elem_bxy, images_size=[264, 264, 1]):        
    xs = [j["x"] for j in elem_bxy]
    ys = [j["y"] for j in elem_bxy]
    x_min = min(xs) #- (max(xs) - min(xs))*0.1
    y_min = min(ys) #- (max(ys) - min(ys))*0.1
    x_max = max(xs) #+ (max(xs) - min(xs))*0.1
    y_max = max(ys) #+ (max(ys) - min(ys))*0.1
    return [{'size': images_size, "x_min": max(1, x_min), "y_min": max(1, y_min), "x_max": min(images_size[1], x_max), "y_max": min(images_size[1], y_max)}]

In [22]:
# Open the dictionary with pandas (Key values: name of the image (42.6339681_2.8641433.png), 
#                                  Values: has_pool:boolean, center: dict(lat,lon), bounds_lat_lon: dict(lat,lon), 
#                                          bounds_x_y: dict(x,y))

df_metadata = pd.read_json(metadata_path, orient="value")
df_metadata = df_metadata.T

images_name = os.listdir(images_dir_path)                              
img_save = cv2.imread(os.path.join(images_dir_path, images_name[10]))  # We take the size of a sample image, this works because all images have the same size

df_metadata["bounds_square"] = df_metadata["bounds_x_y"].apply(lambda x: get_squares(x, img_save.shape))       # We got the size of the of the image previous
df_metadata.head()

,has_pool,center,bounds_lat_lon,bounds_x_y,bounds_square
42.7363609_2.8868181.png,True,"{'lat': 42.7363609, 'lon': 2.8868181}","[{'lat': 42.7364046, 'lon': 2.8867693}, {'lat'...","[{'x': 219.88854424638464, 'y': 197.9724640370...","[{'size': (512, 512, 3), 'x_min': 199.24285541..."
42.7008031_2.906998.png,True,"{'lat': 42.7008031, 'lon': 2.9069979999999997}","[{'lat': 42.700821, 'lon': 2.9069484}, {'lat':...","[{'x': 219.31757991040274, 'y': 223.9864984406...","[{'size': (512, 512, 3), 'x_min': 217.02492867..."
42.6892307_2.9110119.png,True,"{'lat': 42.6892307, 'lon': 2.9110119}","[{'lat': 42.6891823, 'lon': 2.9110157}, {'lat'...","[{'x': 258.80982304681845, 'y': 290.6887228406...","[{'size': (512, 512, 3), 'x_min': 215.62727932..."
44.5015552_0.1670612.png,True,"{'lat': 44.5015552, 'lon': 0.1670612}","[{'lat': 44.5015798, 'lon': 0.1670115999999999...","[{'x': 218.20287511855798, 'y': 215.7165834111...","[{'size': (512, 512, 3), 'x_min': 218.20287511..."
43.3741395_3.3383116.png,True,"{'lat': 43.3741395, 'lon': 3.3383116}","[{'lat': 43.3742033, 'lon': 3.3383698}, {'lat'...","[{'x': 299.5176070846709, 'y': 176.37075727338...","[{'size': (512, 512, 3), 'x_min': 177.63839821..."


In [8]:
# Create the 3 basic folder in ResNet

Path(retnet_path / "Pools").resolve().mkdir(parents=True, exist_ok=True)
pools_path = retnet_path / "Pools"
Path(pools_path / "Annotations").resolve().mkdir(parents=True, exist_ok=True)
Path(pools_path / "ImageSets").resolve().mkdir(parents=True, exist_ok=True)
Path(pools_path / "JPEGImages").resolve().mkdir(parents=True, exist_ok=True)

In [9]:
# Selected the images for training, validation and test

random.seed(1)
old_images_name = images_name
random.shuffle(images_name)
s1 = int(len(images_name)*0.7)
s2 = s1 + int((len(images_name)-s1)/2)

train_imgs = images_name[:s1]
val_imgs = images_name[s1:s2]
test_imgs = images_name[s2:]

In [15]:
# Tranform the images from png to jpeg

if os.listdir(pools_path / "JPEGImages") == [] or ".ipynb_checkpoints" in os.listdir(pools_path / "JPEGImages"):
    path_to_jpeg = pools_path / "JPEGImages"
    i = 1
    total = len(old_images_name)
    for image in old_images_name:
        print(f"\rTransformimg image {image} {i}/{total} ", end="")
        im = Image.open(images_dir_path / image)
        rgb_im = im.convert('RGB')
        filename = str(image)[:-4] + ".jpg"
        rgb_im.save(path_to_jpeg / filename)
        i+=1
else:
    if len(os.listdir(pools_path / "JPEGImages")) > 10:
        print(str(os.listdir(pools_path / "JPEGImages")[:10],) + ' ...')
    else:
        print(os.listdir(pools_path / "JPEGImages"),)
    pass

Transformimg image 42.7028603_2.8301305.png 14912/14912  

In [16]:
# Check if the conversaion if fine

print(len(os.listdir(pools_path / "JPEGImages")), len(old_images_name))

14912 14912


In [29]:
# Annotate the images in the Annotations folder

path_to_jpeg = pools_path / "JPEGImages"
i = 1
total = len(old_images_name)
for image in old_images_name:
    filename = str(image)[:-4] + ".xml"
    print(f"\rAnnotating image {image} {i}/{total} ", end="")
    with open(os.path.join(pools_path,"Annotations/",filename), "w") as f:
        image_jpg = str(image)[:-4] + ".jpg"
        f.flush()
        line = '<annotation verified="yes">\n\t<folder>images</folder>\n\t<filename>'+ image_jpg + '</filename>\n\t'
        f.write(line)
        line = '<path>'+ str(path_to_jpeg) + '\\' + image_jpg +'</path>\n\t<source>\n\t\t<database>Unknown</database>\n\t</source>\n\t'
        f.write(line)
        line = '<size>\n\t\t' + '<width>'+ str(df_metadata.loc[image]["bounds_square"][0]["size"][0]) +'</width>\n\t\t'
        f.write(line)
        line = '<height>'+ str(df_metadata.loc[image]["bounds_square"][0]["size"][1]) +'</height>\n\t\t' 
        f.write(line)
        line = '<depth>'+ str(df_metadata.loc[image]["bounds_square"][0]["size"][2]) +'</depth>\n\t</size>\n\t'
        f.write(line)
        line = '<segmented>0</segmented>\n\t<object>\n\t\t<name>swimmingpool</name>\n\t\t<pose>Unspecified</pose>\n\t\t'
        f.write(line)
        line = '<truncated>0</truncated>\n\t\t<difficult>0</difficult>\n\t\t<bndbox>\n\t\t\t'
        f.write(line)
        line = '<xmin>'+ str(int(df_metadata.loc[image]["bounds_square"][0]["x_min"])) +'</xmin>\n\t\t\t' 
        if (int(df_metadata.loc[image]["bounds_square"][0]["x_min"])) <0:
            print(int(df_metadata.loc[image]["bounds_square"][0]["x_min"]), image_jpg)
        f.write(line)
        line = '<ymin>'+ str(int(df_metadata.loc[image]["bounds_square"][0]["y_min"])) +'</ymin>\n\t\t\t'
        f.write(line)
        line = '<xmax>'+ str(int(df_metadata.loc[image]["bounds_square"][0]["x_max"])) +'</xmax>\n\t\t\t' 
        f.write(line)
        line = '<ymax>'+ str(int(df_metadata.loc[image]["bounds_square"][0]["y_max"])) +'</ymax>\n\t\t'
        f.write(line)
        line = '</bndbox>\n\t</object>\n</annotation>'
        f.write(line)
    i+=1


Annotating image 42.7028603_2.8301305.png 14912/14912  

In [18]:
# Separated images in training, validation and test sets

Path(pools_path / "ImageSets" / "Main").resolve().mkdir(parents=True, exist_ok=True)

text_path = pools_path / "ImageSets" / "Main"
trainval_images = train_imgs + val_imgs

if os.listdir(text_path) == [] or ".ipynb_checkpoints" in os.listdir(text_path):
    with open(text_path/"train.txt", "w") as f:
        for image in train_imgs:
            image_jpg = str(image)[:-4]
            line = image_jpg + "\n"
            f.write(line)
    with open(text_path/"trainval.txt", "w") as f:
        for image in trainval_images:
            image_jpg = str(image)[:-4]
            line = image_jpg + "\n"
            f.write(line)
    with open(text_path/"val.txt", "w") as f:
        for image in val_imgs:
            image_jpg = str(image)[:-4]
            line = image_jpg + "\n"
            f.write(line)
else:
    print(os.listdir(text_path))
    pass

In [9]:
# Create the folder for the output of th training

Path(retnet_path / "OutputTraining").resolve().mkdir(parents=True, exist_ok=True)

out_train = retnet_path / "OutputTraining"

Path(out_train / "TrainOut").resolve().mkdir(parents=True, exist_ok=True)
Path(out_train / "TrainOut" / "snapshots").resolve().mkdir(parents=True, exist_ok=True)
Path(out_train / "TrainOut" / "train").resolve().mkdir(parents=True, exist_ok=True)


------------------------------------------------------------------
To train the model, we moved to the OutputTraining folder and run the following command in powershell
 - **python D:/Avelyn/AvyTareas/DSleuven/Advanced_Analytics_in_Business/Assignment_2/Project/RetNet/keras-retinanet/keras_retinanet/bin/train.py --tensorboard-dir ./TrainOut --snapshot-path ./TrainOut/snapshots --random-transform --steps 100 pascal D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project\RetNet\Pools** <br>

For reproduction, someone would need to change the following paths
 - path to the train.py script in keras-retinanet
 - path to the folder with you want to save the performance of the model (In this example. ./TrainOut) and the model (In this example. ./TrainOut/snapshots)
 - path to the parent folder of JPEGImages, Annotations and ImageSet (In this example D:\Avelyn\AvyTareas\DSleuven\Advanced_Analytics_in_Business\Assignment_2\Project\RetNet\Pools)
 
To evaluate the performance of the training, run the following command in the folder that you defined as tensorboard-dir path before
 - **tensorboard --logdir ./**
 --------------------------------------------------------

In [11]:
# Create the folder to the test output

Path(pools_path / "OutputTest").resolve().mkdir(parents=True, exist_ok=True)

In [27]:
# Prepare the csv of 100 images from the test images to add and a list of extra images to test

test1 = []

for aux in test_imgs[:100]:
    test1.append(aux[:-4] + ".jpg")

aux_dict = {"image_name": test1}
aux_df = pd.DataFrame(aux_dict)

path_tstcsv = pools_path / "OutputTest" / "testimages.csv"
aux_df.to_csv(path_tstcsv, index=False)

print(len(os.listdir(pools_path / "OutputTest")))

test2 = os.listdir(pools_path / "JPEGImagesExtra")

102


## Test evaluation
**To run a detection on the test images, we edited script testDetection.py, setting the appropiate paths and run it.**

---------------------------------------------------------

### Extra code for report purpose

In [13]:
# Draw the boundary polygon for images with doubious quality
t_images = ["43.6488262_1.4292858.png", "42.6367168_2.6882326.png", "42.6371408_2.8691249.png", "42.637546_2.689366.png", 
            "42.6382407_2.8750289.png", "42.6383866_2.8664744.png", "42.6390448_2.867636.png", "42.6396704_2.8179006.png", 
            "42.6398678_2.7250487.png", "42.6399892_2.8665618.png", "42.6405307_2.725142.png"]

print("Images in metadata: ", len(df_metadata))
print("Images in images: ", len(os.listdir(images_dir_path)))

Path(base_path / "BadImages").resolve().mkdir(parents=True, exist_ok=True)
bi_path = base_path / "BadImages"
n_images = len(t_images)#len(images_name)             # How many images you want to display    

for i in range(n_images):
        img_save = cv2.imread(os.path.join(images_dir_path, t_images[i]))
        pts = np.array( [list((j["x"],j["y"])) for j in df_metadata.loc[t_images[i]]["bounds_x_y"]], np.int32 )
        pts = pts.reshape(-1,1,2)
        cv2.polylines(img_save,[pts],True,(0,0,255), 2);
        abs_path_save = os.path.join(bi_path, t_images[i])
        cv2.imwrite(abs_path_save, img_save)

Images in metadata:  14964
Images in images:  14912
